In [1]:
import quadprog
import qpsolvers
import numpy

This program accepts X, a matrix whose rows are the samples, and y, a vector whose elements are the target values.  The goal is to model the data using an SVM or Support Vector Machine.  The program uses quadratic programming, which is a means of finding the maximum value of a convex (downward pointing) quadratic function given a set of linear inequalities and linear constraints.  For instance, in one dimension we might find the maximum of an upside-down parabola given the linear constraint x > 10.  However, the SVM is described by a set of equations that do not readily translate into a quadratic programming problem.  Therefore, we must utilize some heavy-duty math to translate the SVM problem into a "dual problem" which is amenable to quadratic programming.  We solve for a sequence of "alpha" values that correspond to the dual problem.  The SVM program then finds a w and b value, which can be used to make predictions via w.dot(x) + b > 0 vs. < 0, thereby classifying x.

The goal is to minimize $(1/2)x^T P x + q^T x$ subject to  $G x \le h$ and $A x = b$.

So we want:

x = alphas

P = a matrix of kernel values (inner products between sample vectors) which are multiplied by 1 if the sample vectors are in the same class; otherwise by -1

q = a vector of N 1's, whose length equals the number of samples.

G = the identity matrix.

h = a vertical vector of all 0's, whose length equals the number of samples.  These values of G and h together result in the constraint that all alphas are greater than zero.

A = a vector of N y's (1s and -1s).  (Maybe it should say A^T x = b.)

b = a scalar 0

We will do the prediction using a kernel function. The basic kernel function is x1.dot(x2), but that can also be squared; or a constant can be added and then it's squared, for instance. When the kernel is the square of a dot product, it represents an N^2 dimensional space where the values are the products of pairs of values in the x1 vector.  If x1 has dimension N, then the space of products of pairs of its values has dimension N^2, although the manifold of allowed values within that space still has dimension N.

In [ ]:
def kernel(x1, x2):
    return (x1.dot(x2) + 1000000.)**2 / 1000000000.

The kernel matrix finds the kernel value for each pair of rows of X; this becomes a matrix.

In [2]:
def kernel_matrix(X):
  return numpy.fromfunction(numpy.vectorize(lambda i, j: kernel(X[int(i)], X[int(j)])), (X.shape[0], X.shape[0]))

Here is some old data that could be input to the quadratic programming solver to find alpha. It represents a manifold of the from [x, x\*\*2] with x taking on values in [-2, -1, 0, 1, 2]

In [3]:
# X is the samples
# X = numpy.array([[0.0, 0.0], [1.0, 1.0], [-1.0, 1.0], [2.0, 4.0], [-2.0, 4.0]])
# y is the target value for each sample
# y = numpy.array([-1.0, -1.0, 1.0, 1.0, 1.0])
# ker = kernel_matrix(X)
# We have to add an epsilon to make P be positive definite instead of just
# nonnegative definite
# P = numpy.multiply(ker, numpy.outer(y, y)) + numpy.eye(X.shape[0]) * 0.00001
# q = -numpy.ones(X.shape[0])
# G = -numpy.eye(X.shape[0])
# h = numpy.zeros(X.shape[0])
# A = y
 #b = numpy.array([0.0])
# alpha = numpy.round(qpsolvers.solve_qp(P, q, G, h, A, b, solver = "quadprog"), 2)

This function computes the value of b, the intercept in the expression w^T x + b that is used to define the decision boundary where we want to find the least magniture possible weight vector w such that $y^{(i)} (w^T x^{(i)} + b) \ge 1$ for all i.

In [436]:
def compute_b(ker, y, alpha):
  mx = numpy.NINF
  mn = numpy.inf
  for i in range(ker.shape[0]):
    mx = max(mx, sum(alpha[j] * y[j] * ker[i, j] for j in range(ker.shape[0]))) if y[i] == -1 else mx
    mn = min(mn, sum(alpha[j] * y[j] * ker[i, j] for j in range(ker.shape[0]))) if y[i] == 1 else mn
  return -(mx + mn)/2

This is the formula for predicting using a kernel, from p. 13 of the Stanford CS 229 lecture by Andrew Ng.

In [437]:
def pred_kernel(ker, X, y, alpha, x, b):
  return numpy.round(sum(alpha[i] * y[i] * kernel(X[i],x) for i in range(ker.shape[0])) + b, 3)

Now I will apply the above functions to MNIST data.

In [438]:
from sklearn.datasets import fetch_mldata

In [439]:
mnist = fetch_mldata("MNIST original")

In [440]:
mnist.target

array([0., 0., 0., ..., 9., 9., 9.])

SVM doesn't scale all that well with the number of samples.  Here, I will just use a random 6,000 training samples.  I can't use the _first_ 6,000 samples in MINST, because they all have a target value of zero.  Here, I am doing a one vs. all test, checking whether a given number is zero or not.  To do a more universal check, I would have to make ten different one vs. all tests or 45 one vs. one tests.

In [467]:
import random

nsamples = 6000

indices = random.sample(range(1, 60000), nsamples)
# X is the samples
X = mnist.data[indices].astype(float)
# y is the target value for each sample
y = ((mnist.target[indices] == 0) * 2.0 - 1.0).astype(float)
ker = kernel_matrix(X)
# We have to add an epsilon to make P be positive definite instead of just
# nonnegative definite
P = numpy.multiply(ker, numpy.outer(y, y)) + numpy.eye(nsamples) * 1000000. #0.00001
q = -numpy.ones(nsamples)
G = -numpy.eye(nsamples)
h = numpy.zeros(nsamples)
A = y
b = numpy.array([0.0])
# alpha = qpsolvers.solve_qp(P, q, G, h, A, b, solver = "CVXOPT")

In [468]:
ker

array([[27611.11541838,  6990.741288  ,  3629.5431785 , ...,
         5006.28542573,  6369.37968017,  7970.67845112],
       [ 6990.741288  , 45597.56624441,  2882.6470829 , ...,
         9919.71559536,  6154.54229722,  8974.5779776 ],
       [ 3629.5431785 ,  2882.6470829 , 11400.62394336, ...,
         5853.3221645 ,  3295.15071502,  3753.03442563],
       ...,
       [ 5006.28542573,  9919.71559536,  5853.3221645 , ...,
        46789.44415896, 13918.40602464, 20096.47310228],
       [ 6369.37968017,  6154.54229722,  3295.15071502, ...,
        13918.40602464, 54098.58457028, 14854.0097281 ],
       [ 7970.67845112,  8974.5779776 ,  3753.03442563, ...,
        20096.47310228, 14854.0097281 , 66783.85583532]])

In [469]:
alpha = qpsolvers.solve_qp(P, q, G, h, A, b, solver = "quadprog")

In [470]:
alpha

array([4.42169376e-25, 3.04549590e-07, 4.99499866e-08, ...,
       7.16446049e-08, 1.10909317e-08, 1.91659718e-21])

In [471]:
b = compute_b(ker, y, alpha)

In [472]:
for n in list(range(60000, 60010)) + list(range(65000, 65010)):
    inp = mnist.data[n].astype(float)
    pred = pred_kernel(ker, X, y, alpha, inp, b)

    print("Prediction: ", pred, bool((pred / numpy.abs(pred) + 1.0) / 2.0) if pred != 0.0 else True, "Actual:", mnist.target[n] == 0)

Prediction:  1.144 True Actual: True
Prediction:  0.895 True Actual: True
Prediction:  0.972 True Actual: True
Prediction:  2.078 True Actual: True
Prediction:  1.206 True Actual: True
Prediction:  0.533 True Actual: True
Prediction:  0.71 True Actual: True
Prediction:  1.835 True Actual: True
Prediction:  0.859 True Actual: True
Prediction:  0.322 True Actual: True
Prediction:  -1.683 False Actual: False
Prediction:  -1.014 False Actual: False
Prediction:  -0.825 False Actual: False
Prediction:  -0.54 False Actual: False
Prediction:  -0.651 False Actual: False
Prediction:  -0.733 False Actual: False
Prediction:  -0.494 False Actual: False
Prediction:  -0.639 False Actual: False
Prediction:  -0.581 False Actual: False
Prediction:  -0.596 False Actual: False


Predict all 10,000 "test" items in MNIST.

In [473]:
TP = 0
FP = 0
FN = 0
TN = 0

for n in list(range(60000, 70000)):
    inp = mnist.data[n].astype(float)
    pred = pred_kernel(ker, X, y, alpha, inp, b)
    pred = bool((pred / numpy.abs(pred) + 1.0) / 2.0) if pred != 0.0 else True
    actual = (mnist.target[n] == 0)
    if pred and actual:
        TP += 1
    if pred and not actual:
        FP +=1
    if not pred and actual:
        FN +=1
    if not pred and not actual:
        TN +=1

print(TP, FP, FN, TN)

959 193 21 8827


In [475]:
(959 + 8827)/10000

0.9786